In [13]:
from openai import OpenAI
import os
# pip install openai



In [14]:
# insert your api key here
client = OpenAI(api_key="sk-proj-")


In [15]:
# Creating vector storage
vector_store = client.vector_stores.create(name="my_rag_store")


In [16]:
# 2) Upload files and link it to the vector storage
file_paths = ["/Users/d2m/Downloads/electric_kettle_manual.txt"] # insert your path here
file_ids = []
for path in file_paths:
    file_obj = client.files.create(
        file=open(path, "rb"),
        purpose="assistants"
    )
    file_ids.append(file_obj.id)
    
# Linking files to vector storage
for f_id in file_ids:
    client.vector_stores.files.create(
        vector_store_id=vector_store.id,
        file_id=f_id
    )

print("Vector store id:", vector_store.id)



Vector store id: vs_6924232249c0819198fe49222686271a


In [19]:
VECTOR_STORE_ID = vector_store.id 

def call_gpt(prompt: str):
    response = client.responses.create(
        model="gpt-4o-mini",
        input=prompt,
        temperature=0.0, # param to turn off context outside files inside storage
        tools=[
            {
                "type": "file_search",
                "vector_store_ids": [VECTOR_STORE_ID],
                "max_num_results": 10,  # tuning param
            }
        ]
    )
    return response.output_text



In [20]:
# Response based on uploaded file only
question = "Make a summary of the contents."
prompt = (
        "Answer ONLY based on the documents from the connected RAG corpus. "
        "If the required information is not present in the documents, write you don't have information.\n\n"
        f"Question: {question}"
    )

print(call_gpt(prompt))


The document is an **Electric Kettle User Manual** that provides essential information on using the kettle safely and efficiently. Here’s a summary of its contents:

1. **Introduction**: Overview of the kettle's purpose for fast water boiling.
2. **Package Contents**: Lists items included with the kettle.
3. **Safety Instructions**: Guidelines to prevent accidents, such as not immersing the kettle in water and ensuring it is not turned on without water.
4. **How to Use**: Step-by-step instructions for operating the kettle.
5. **Cleaning and Maintenance**: Recommendations for regular descaling and cleaning methods.
6. **Technical Specifications**: Details on power, capacity, and materials.
7. **Common Issues and Solutions**: Troubleshooting tips for common problems.
8. **Warranty**: Information about the standard one-year warranty.

The manual aims to ensure safe and effective use of the electric kettle.


In [ ]:
# Manual content used to generete response
'''
Electric Kettle User Manual

1. Introduction

This simple guide will help you quickly understand how to use your
electric kettle safely and efficiently. The kettle is designed for fast
water boiling in everyday household use.

2. Package Contents

-   Electric kettle
-   Power base
-   Quick start manual (you’re reading the extended version 🙂)

3. Safety Instructions

-   Do not immerse the kettle in water or any other liquid.
-   Do not turn it on without water inside.
-   Keep it away from children and pets.
-   Always use a functioning power outlet.

4. How to Use

1.  Place the kettle on a flat surface.
2.  Remove it from the base and fill it with water within the MIN/MAX
    markings.
3.  Put the kettle back on the base.
4.  Switch it on by pressing the power lever.
5.  Wait for the automatic shut‑off after boiling.

5. Cleaning and Maintenance

-   Descale the kettle regularly (every 1–2 months).
-   Use citric acid solution or a descaling agent.
-   Do not use abrasive cleaning materials.

6. Technical Specifications

-   Power: 1800–2200 W
-   Capacity: 1.5–2 L
-   Body material: glass/plastic/stainless steel (depending on model)

7. Common Issues and Solutions

-   Won’t turn on: check the outlet and ensure the kettle is seated
    properly on the base.
-   Boils slowly: descaling may be needed.
-   Strange taste: rinse the kettle several times with clean water.

8. Warranty

Standard warranty — 1 year from the date of purchase. Keep your receipt
for service.

Thank you for using our kettle! May your water boil fast and your mood
stay warm ☕
'''